This notebook generates videos (as .GIF files) for the visual search trajectories of each subject in each trial in each choice set size. 

Momentary fixation positions are marked by a white square, while choices are indicated by the red square. For better visibility, all fixation durations have been doubled. The resulting .GIF files are stored to '../results/exemplar_visual-search-trajectories/'. 

Animations 1-4 of the manuscript are from the following combination of set size, subject, and trial:
- Animation 1: set size: 9, subject: 11, trial: 8
- Animation 2: set size: 16, subject: 0, trial: 76
- Animation 3: set size: 25, subject: 7, trial: 137
- Animation 4: set size: 36, subject: 37, trial: 24

In [ ]:
from pathlib import Path
import os
import shutil
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw

import matplotlib.pyplot as plt
%matplotlib inline

from utils.general import make_sure_path_exists

# Descriptives

In [2]:
# set sizes & subjects:
setsizes = [9, 16, 25, 36]
subjects = np.arange(49)

# directories
data_dir = '../data/'
results_dir = '../results/'
video_dir = results_dir+'exemplar_visual-search-trajectories/'
make_sure_path_exists(video_dir)

# read out stimuli
stimulus_dir = data_dir+'stimuli/'
stimuli = [s for s in os.listdir(stimulus_dir)]

In [3]:
def make_fix_screen(r=20):
    """
    make fixation cross screen
    """
    # black background
    screen = Image.new(mode='RGBA', size=(1280, 1024), color='black')
    # add fix cross
    fix_draw = ImageDraw.Draw(screen) 
    screen_center = np.array([1280, 1024])/2
    # horizontal
    fix_draw.line((screen_center[0]-r, screen_center[1],
                   screen_center[0]+r, screen_center[1]),
                  fill='white', width=5)
    # vertical
    fix_draw.line((screen_center[0], screen_center[1]-r,
                   screen_center[0], screen_center[1]+r),
                  fill='white', width=5)
    return screen

In [4]:
def make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir):
    """
    make choice set screen
    """
    # black background
    screen = Image.new(mode='RGBA', size=(1280, 1024), color='black')
    for i, stimfile in enumerate(trial_stimuli):
        # get stim pos
        stimbox = stimulus_positions.loc[i].values.astype(np.int)
        # get target size
        stimsize = (stimbox[2]-stimbox[0], stimbox[3]-stimbox[1])
        # load stim img
        stimimg = Image.open(stimulus_dir+stimfile)
        # resize stim
        stimimg = stimimg.resize(size=stimsize)
        # paste
        screen.paste(stimimg, box=stimbox)
    return screen

In [5]:
def draw_gaze_marker(screen, pos, color='white', width=5):
    """
    add current gaze position to screen
    """
    # init draw
    box_draw = ImageDraw.Draw(screen)
    # horizontal left
    box_draw.line((pos[0], pos[1],
                   pos[2], pos[1]),
                  fill=color, width=width)
    # horizontal right
    box_draw.line((pos[0], pos[3],
                   pos[2], pos[3]),
                  fill=color, width=width)
    # vertical top
    box_draw.line((pos[0], pos[1],
                   pos[0], pos[3]),
                  fill=color, width=width)
    # vertical bottom
    box_draw.line((pos[2], pos[1],
                   pos[2], pos[3]),
                  fill=color, width=width)
    return screen

In [6]:
# generate visual search GIF for each empirical trial

np.random.seed(1)

for setsize in setsizes:
    # load data
    setsize_data = pd.read_csv(data_dir+'summary_files/{}_data.csv'.format(setsize))
    stimulus_positions = pd.read_csv(data_dir+'stimulus_positions/{}_stimulus_positions.csv'.format(setsize), header=None)
    stimulus_centers = np.concatenate(((stimulus_positions.loc[:,0].values+0.5*(stimulus_positions.loc[:,2].values-stimulus_positions.loc[:,0].values))[:,None],
                                       (stimulus_positions.loc[:,1].values+0.5*(stimulus_positions.loc[:,3].values-stimulus_positions.loc[:,1].values))[:,None]), axis=1)
    for subject in subjects:
        # subset & load data
        subject_data = setsize_data[setsize_data['subject']==subject].copy()
        subject_gaze_data = pd.read_csv(data_dir+'subject_files/{}_{}_fixations.csv'.format(subject, setsize))
        # get trials
        trials = subject_gaze_data.trial.unique()
        for trial in trials:
            # define filepath
            gif_filepath = video_dir+'setsize-{}_sub-{}_trial-{}.gif'.format(setsize, subject, trial)
            if not os.path.isfile(gif_filepath):
                print('Processing: setsize: {}, subject: {}, trial: {}'.format(setsize, subject, trial))
                # subset data
                trial_data = subject_data[subject_data['trial']==trial].copy()
                trial_gaze_data = subject_gaze_data[subject_gaze_data['trial']==trial].copy()
                # get trial stimuli
                trial_stimuli = trial_data[['stimulus_{}'.format(i) for i in range(setsize)]].values[0]
                # make fix screen
                fix_screen = make_fix_screen()
                # make choice screen
                trial_img = [fix_screen]
                for fi, fixitem in enumerate(trial_gaze_data['item'].values.astype(np.int)):
                    # make choice screen
                    fix_screen = make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir)
                    # add fixation markers
                    fix_screen = draw_gaze_marker(fix_screen, stimulus_positions.loc[fixitem])
                    # collect
                    trial_img.append(fix_screen)
                # add choice indication
                choice_screen = make_choice_screen(trial_stimuli, stimulus_positions, stimulus_dir)
                choice_screen = draw_gaze_marker(choice_screen,
                                                 stimulus_positions.loc[trial_data['choice'].values[0]],
                                                 color='red', width=5)
                trial_img.append(choice_screen)
                # make gif & save
                trial_img[0].save(gif_filepath, save_all=True, optimize=False, append_images=trial_img[1:], loop=0, 
                                  duration=[500*2]+list(trial_gaze_data['dur'].values.astype(np.int)*2)+[500*2])

Processing: setsize: 9, subject: 5, trial: 112
Processing: setsize: 9, subject: 5, trial: 113
Processing: setsize: 9, subject: 5, trial: 116
Processing: setsize: 9, subject: 5, trial: 118
Processing: setsize: 9, subject: 5, trial: 120
Processing: setsize: 9, subject: 5, trial: 123
Processing: setsize: 9, subject: 5, trial: 124
Processing: setsize: 9, subject: 5, trial: 130
Processing: setsize: 9, subject: 5, trial: 139
Processing: setsize: 9, subject: 5, trial: 146
Processing: setsize: 9, subject: 5, trial: 151
Processing: setsize: 9, subject: 5, trial: 156
Processing: setsize: 9, subject: 5, trial: 158
Processing: setsize: 9, subject: 5, trial: 160
Processing: setsize: 9, subject: 5, trial: 162
Processing: setsize: 9, subject: 5, trial: 169
Processing: setsize: 9, subject: 5, trial: 172
Processing: setsize: 9, subject: 5, trial: 176
Processing: setsize: 9, subject: 5, trial: 181
Processing: setsize: 9, subject: 5, trial: 185
Processing: setsize: 9, subject: 5, trial: 187
Processing: s

KeyboardInterrupt: 